In [1]:
import sys
import os
from scipy.sparse import load_npz
sys.path.append('./codes/forgraph/')
from config import args
from sklearn.metrics import roc_auc_score
from models import GCN
from metrics import *
import numpy as np
import pandas as pd
import scipy
#import torch_geometric
from scipy.sparse import coo_matrix,csr_matrix
import pickle as pkl
import csv
import torch
from Explainer import Explainer

In [2]:
def acc(adj,insert):
    mask = explainer.masked_adj.numpy()
    adj = coo_matrix(adj)
    for r,c in list(zip(adj.row,adj.col)):
        if r>=insert and r<insert+skip and c>=insert and c<insert+skip:
            reals.append(1)
        else:
            reals.append(0)
        preds.append(mask[r][c])
        
def explain_graph(gid):
    fea,emb,adj,label,graphid = features[gid], embs[gid], adjs[gid], labels[gid], gid
    explainer((fea,emb,adj,1.0,label))
    insert = 20
    acc(adj,insert)
    
def test():
    global preds
    global reals
    preds = []
    reals = []
    for gid in allnodes:
        explain_graph(gid)
    auc = roc_auc_score(reals,preds)
    subset_acc = auc
    all_node_acc = roc_auc_score(reals, allnodes)
    fidelity = all_node_acc - subset_acc
    subset_nodes = len(preds)
    all_nodes = len(allnodes)
    sparsity = 1 - subset_nodes / all_nodes
    return auc, fidelity, sparsity

def train(epochs):
    t0 = args.coff_t0
    t1 = args.coff_te

    for epoch in range(epochs): 
        loss = 0
        tmp = float(t0 * np.power(t1 / t0, epoch /epochs))
        train_instances = [ins for ins in range(len(model.values()))]
        np.random.shuffle(train_instances)
        for gid in train_instances:
            with tf.GradientTape() as tape:
                pred = explainer((features[gid],embs[gid],adjs[gid],tmp, labels[gid]),training=True)
                loss += explainer.loss(pred, labels[gid])
        train_variables = [para for para in explainer.trainable_variables]
        grads = tape.gradient(loss, train_variables)
        optimizer.apply_gradients(zip(grads, train_variables))

In [3]:
# file = 'actual data/src/data/E116/saved_runs/model_2020-12-17-at-19-24-36.pt'
#file = 'actual data/src/data/E116/df_genes.pkl'
#model = torch()
#model.load_state_dict(torch.load(file))
# model = torch.load(file, map_location=torch.device('cpu'))

In [4]:
cell_line = 'E116'
base_path = 'actual data/src/data'
hic_sparse_mat_file = os.path.join(base_path, cell_line, 'hic_sparse.npz')
np_nodes_lab_genes_file = os.path.join(base_path, cell_line, 'np_nodes_lab_genes.npy')
np_hmods_norm_all_file = os.path.join(base_path, cell_line, 'np_hmods_norm.npy') 
df_genes_file = os.path.join(base_path, cell_line, 'df_genes.pkl')
df_genes = pd.read_pickle(df_genes_file)

In [5]:
df_genes

,gene_catalog_name,abbrev,hic_node_id,expression_lvl,connected
0,ENSG00000237613,FAM138A,3,0,1.0
1,ENSG00000186092,OR4F5,6,0,1.0
2,ENSG00000235249,OR4F29,36,0,1.0
3,ENSG00000185097,OR4F16,62,0,1.0
4,ENSG00000197049,NaN,72,1,1.0
...,...,...,...,...,...
384,ENSG00000100288,CHKB,279585,0,1.0
385,ENSG00000100299,ARSA,279588,1,1.0
386,ENSG00000251322,SHANK3,279593,0,1.0
387,ENSG00000100312,ACR,279599,0,1.0


In [6]:
mat = load_npz(hic_sparse_mat_file)
allNodes_hms = np.load(np_hmods_norm_all_file)
hms = allNodes_hms[:, 1:] #only includes features, not node ids
allNodes = allNodes_hms[:, 0].astype(int)
geneNodes_labs = np.load(np_nodes_lab_genes_file)
geneNodes = geneNodes_labs[:, -2].astype(int)
geneLabs = geneNodes_labs[:, -1].astype(int)

In [7]:
allLabs = 2*np.ones(np.shape(allNodes))
allLabs[geneNodes] = geneLabs

In [8]:
features = hms
adjs = mat
labels = allLabs

In [9]:
all_nodes = len(labels)

In [10]:
all_nodes

279606

In [11]:
print(features.shape)
print(labels.shape)
print(adjs.shape)
print(features.shape[-1])

(279606, 5)
(279606,)
(279606, 279606)
5


In [12]:
#features = scipy.sparse.csr_matrix(features)

In [36]:
embs = scipy.sparse.hstack([features, adjs])

In [14]:
#arr.shape

In [15]:
#type(arr)

In [16]:
#embs = tf.convert_to_tensor(arr, dtype=tf.float32)

In [17]:
# model = GCN(input_dim = adjs.shape[-1], output_dim = labels.shape[0])

In [18]:
features.shape

(279606, 5)

In [19]:
adjs.shape

(279606, 279606)

In [20]:
t = np.transpose(features)
t.shape

(5, 279606)

In [21]:
m = t * adjs
m.shape

(5, 279606)

In [22]:
n = adjs * features
n.shape

(279606, 5)

In [23]:
np.matmul(adjs, features)

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [25]:
np.matmul(features, adjs)

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [26]:
# model = GCN(input_dim = adjs.shape[-1], output_dim = labels.shape[0])
# embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
#                                tf.convert_to_tensor(scipy.sparse.csr_matrix.getnnz(adjs),dtype=tf.float32)), training=False)
# outputs = model.call((tf.convert_to_tensor(features,dtype=tf.float32),\
#                             tf.convert_to_tensor(scipy.sparse.csr_matrix.getnnz(adjs),dtype=tf.float32)),training=False)
# outputs = model.call(embs)
# pred_label = tf.argmax(outputs, 1)
# acc = accuracy(outputs, labels)
# print('model acc',acc.numpy())

In [27]:
# embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
#                                tf.convert_to_tensor(adjs,dtype=tf.float32)), training=False)
# rows = features.shape[0]
# cols_f = features.shape[1]
# cols_a = adjs.shape[1]
# a = []
# for i in range(0, rows):
#     lst = []
#     for j in range(0, cols_f):
#         lst.append(features[i, j])
#     for j in range(0, cols_a):
#         lst.append(adjs[i, j])
#     a.append(lst)
# arr = np.array(a)
# embs = tf.convert_to_tensor(arr, dtype=tf.float32)
# outputs = model.call(embs),training=False)
# pred_label = tf.argmax(outputs, 1)

In [28]:
# if args.setting==1:
#     allnodes = [i for i in range(0,100)]
# elif args.setting==2:
#     allnodes = [i for i in range(0,100)]
#     allnodes.extend([i for i in range(500,600)])
# elif args.setting==3:
#     allnodes=[i for i in range(1000)]
# explainer = Explainer(model=model,nodesize=len(model.values()))

In [29]:
file = 'actual data/src/data/E116/saved_runs/model_2020-12-17-at-19-24-36.pt'
model = torch.load(file, map_location=torch.device('cpu'))

In [30]:
top_k = [i for i in range(0, 26)]
all_elr = [i for i in range(0, 26)]
coff_t0 = [i for i in range(1, 26)]
coff_size = [i for i in range(0, 26)]
coff_ent = [i for i in range(0, 26)]
skp = [i for i in range(0, 26)]
epochs = [i for i in range(250, 650, 50)]

In [31]:
epochs

[250, 300, 350, 400, 450, 500, 550, 600]

In [32]:
rows = []

In [33]:
explainer = Explainer(model=model,nodesize=all_nodes)

"Fidelity metric measures whether the explanations are faithfully important to the model’s predictions. It removes the important structures from the input graphs and computes the difference between predictions. In addition, the Sparsity metric measures the fraction of structures that are identified as important by explanation methods."

In [39]:
values = embs.data
indices = np.vstack((embs.row, embs.col))

i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = embs.shape

embs = torch.sparse.FloatTensor(i, v, torch.Size(shape)).to_dense()

RuntimeError: [enforce fail at C:\Users\builder\tkoch\workspace\pytorch\pytorch_1647970138273\work\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 312723653064 bytes.

In [40]:
count = 0

In [41]:
for k in top_k:
    topk = k
    for elr in all_elr:
        args.elr = elr
        optimizer = tf.keras.optimizers.Adam(learning_rate=args.elr)
        for c in coff_t0:
            args.coff_t0 = c
            for s in coff_size:
                args.coff_size = s
                for en in coff_ent:
                    args.coff_ent = en
                    for sk in skp:
                        skip = sk
                        for ep in epochs:
                            train(ep)
                            auc, fidelity, sparsity = test()
                            row = [k, elr, c, en, sk, ep, auc, fidelity, sparsity]
                            rows.append(row)
                            count += 1
                            if count % 100:
                                print('Ran 100')

TypeError: 'coo_matrix' object is not subscriptable

In [ ]:
col_names = ['K', 'elr', 'coff_t0', 'coff_size', 'coff_ent', 'Skip', 'AUC', 'Fidelity', 'Sparsity']
filename = 'results.csv'
with open('results.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(col_names) 
    write.writerows(rows) 